In [3]:
#Import libraries
import pandas as pd

In [4]:
#Read Data
data = pd.read_csv('vehicles_data.csv')

In [5]:
#Preview Data
data.head()

,Unnamed: 0,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,posting_date
0,0,7307679724,abilene,4500,2002.0,bmw,x5,good,6 cylinders,gas,184000.0,clean,automatic,4wd,full-size,sedan,white,tx,2021-04-16 00:00:00+00:00
1,1,7311833696,abilene,4500,2002.0,bmw,x5,good,6 cylinders,gas,184000.0,clean,automatic,4wd,full-size,sedan,white,tx,2021-04-24 00:00:00+00:00
2,2,7311441996,abilene,4900,2006.0,toyota,camry,excellent,4 cylinders,gas,184930.0,clean,automatic,fwd,full-size,sedan,silver,tx,2021-04-23 00:00:00+00:00
3,3,7307680715,abilene,6500,2008.0,ford,expedition,good,6 cylinders,gas,206000.0,clean,automatic,4wd,full-size,sedan,white,tx,2021-04-16 00:00:00+00:00
4,4,7311834578,abilene,6500,2008.0,ford,expedition,good,6 cylinders,gas,206000.0,clean,automatic,4wd,full-size,sedan,white,tx,2021-04-24 00:00:00+00:00


In [6]:
#delete unncesseccary rows
data.drop(columns=['Unnamed: 0', 'state'], inplace=True)

In [7]:
data.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,posting_date
0,7307679724,abilene,4500,2002.0,bmw,x5,good,6 cylinders,gas,184000.0,clean,automatic,4wd,full-size,sedan,white,2021-04-16 00:00:00+00:00
1,7311833696,abilene,4500,2002.0,bmw,x5,good,6 cylinders,gas,184000.0,clean,automatic,4wd,full-size,sedan,white,2021-04-24 00:00:00+00:00
2,7311441996,abilene,4900,2006.0,toyota,camry,excellent,4 cylinders,gas,184930.0,clean,automatic,fwd,full-size,sedan,silver,2021-04-23 00:00:00+00:00
3,7307680715,abilene,6500,2008.0,ford,expedition,good,6 cylinders,gas,206000.0,clean,automatic,4wd,full-size,sedan,white,2021-04-16 00:00:00+00:00
4,7311834578,abilene,6500,2008.0,ford,expedition,good,6 cylinders,gas,206000.0,clean,automatic,4wd,full-size,sedan,white,2021-04-24 00:00:00+00:00


In [12]:
data.to_csv('new_craiglist_data.csv')

In [8]:
#Convert the new data into sql file
import pandas as pd

# Read the new CSV file into a pandas DataFrame
data = pd.read_csv('new_craiglist_data.csv')
data = data.head(2000)
# Define the table name
table_name = 'vehicles'

# Generate SQL for creating the table
create_table_sql = f"CREATE TABLE vehicles ("
# Iterate through columns to create column definitions
for col in data.columns:
 dtype = data[col].dtype
 if 'int' in str(dtype):
     create_table_sql += f"{col} INT, "
 elif 'float' in str(dtype):
     create_table_sql += f"{col} FLOAT, "
 else:
     create_table_sql += f"{col} VARCHAR(255), "
     create_table_sql = create_table_sql.rstrip(', ') + ");"

# Generate SQL for inserting data
insert_data_sql = f"INSERT INTO vehicles ({', '.join(data.columns)}) VALUES "
for index, row in data.iterrows():
 values = ', '.join([f"'{val}'" if isinstance(val, str) else str(val) for val in row])
 insert_data_sql += f"({values}), "
 insert_data_sql = insert_data_sql.rstrip(', ') + ";"

# Write the SQL statements to a file
with open('vehicless.sql', 'w') as f:
 f.write(create_table_sql + '\n\n')
 f.write(insert_data_sql)

I imported the table into your Sql server ready for uploading.
I created an  Azure blob storage for my work. 

In [ ]:
#I linked the Azure blob storage to my sql server
{
  "name": "craiglistvehicles",
  "properties": {
    "type": "AzureSqlDatabase",
    "typeProperties": {
      "connectionString": "Server=SHERRIF\SQLEXPRESS.database.windows.net;Database=Craiglist;User=DATA@SHERRIF\SQLEXPRESSe;Password=CarelJ2023!@@;Encrypt=true;Connection Timeout=30;"
    }
  }
}


In [ ]:
# Defined source and destination for my data
{
  "name": "vehicless",
  "properties": {
    "mycoollink": {
      "referenceName": "craiglistvehicles",
      "type": "sql"
    },
    "type": "Table",
    "typeProperties": {
      "tableName": "vehicless"
    }
  }
}


In [ ]:
#Created a pipeline to copy data
{
  "name": "Craigcopy",
  "type": "Copy",
  "inputs": [
    {
      "referenceName": "craiglistvehicles",
      "type": "sql"
    }
  ],
  "outputs": [
    {
      "referenceName": "craiglistvehicles",
      "type": "csv"
    }
  ],
  "typeProperties": {
    "source": {
      "type": "SqlSource",
      "sqlReaderQuery": "SELECT * FROM vehicless"
    },
    "sink": {
      "type": "BlobSink",
      "writeBatchSize": 2000,
      "writeBatchTimeout": "60.00:00:00"
    }
  }
}


In [ ]:
#Link azure blob storage to snowfkale to copy data
#one for azure storage
{
  "name": "AzureBlobStorageLinkedService",
  "properties": {
    "type": "AzureBlobStorage",
    "typeProperties": {
      "accountKey": "mykey",
      "accountName": "Acc. name"
    }
  }
}
#one on snowflake
{
  "name": "SnowflakeLinkedService",
  "properties": {
    "type": "Snowflake",
    "typeProperties": {
      "connectionString": "your_snowflake_connection_string"
    }
  }
}


In [ ]:
#Define source and destination for the data, in this case, from azure blod storage to snowflake
{
  "name": "BlobStorageDataset",
  "properties": {
    "linkedServiceName": {
      "referenceName": "craiglistvehicles",
      "type": "csv"
    },
    "type": "AzureBlob",
    "typeProperties": {
      "format": {
        "type": "json"
      },
      "folderPath": "vehicles",
      "fileName": "vehicless.json"
    }
  }
}


In [ ]:
#create data copy pipeline
{
  "name": "CopyData2",
  "type": "Copy",
  "inputs": [
    {
      "referenceName": "craiglistvehicles",
      "type": "json"
    }
  ],
  "outputs": [
    {
      "referenceName": "craiglistvehicles",
      "type": "csv"
    }
  ],
  "typeProperties": {
    "source": {
      "type": "BlobSource",
      "recursive": true
    },
    "sink": {
      "type": "SnowflakeSink",
      "writeBatchSize": 10000,
      "writeBatchTimeout": "60.00:00:00"
    }
  }
}
#Run the pipeline

In [ ]:
#Transform the data using dbt
# Initialize a dbt project
dbt init Craiglist_v

In [ ]:
#configure snowflake connection to my local machine
my_project:
  target: dev
  outputs:
    dev:
      type: snowflake
      account: acc. name
      user: username
      password: mypass
      role: student
      warehouse: craigvehicles
      database: Craig


In [ ]:
#write Dbt models

-- models/vehicless.sql

-- Common Table Expression (CTE) to calculate yearly sales summary
WITH monthly_sales_cte AS (
  SELECT
    DATE_TRUNC('month', sale_date) AS month,
    SUM(sales_amount) AS total_sales
  FROM
    {{ ref('sales') }}
  GROUP BY
    month
)

-- Final SELECT statement to create the transformed table
SELECT
  month,
  total_sales
FROM
  monthly_sales_cte;



In [ ]:
#run execute
dbt run

In [ ]:
#validate the results in snowflake since we did the transformation on our local machine
SELECT * FROM {{ ref('monthly_sales_summary') }};